In [95]:
from keras import Model
from keras.layers import Dense,Flatten,Input,Conv2D,MaxPool2D
from keras.layers import Activation,LeakyReLU,ReLU
from keras.layers.normalization import BatchNormalization

import numpy as np

In [4]:
model_cfg = 'yolov3-tiny.cfg'

In [59]:
def parse_cfg(cfg_path):
    cfg_file = open(cfg_path,'r')
    lines = cfg_file.read().split('\n')
    
    lines = [x for x in lines if x != '']
    lines = [x for x in lines if x[0] != '#']
    
    layers = []
    layer = {}
    
    for line in lines:
        if line[0] == '[':
            if len(layers) != 0 or layer != {}:
                layers.append(layer)
                layer = {}
            layer['type'] = line.replace('[','').replace(']','')
        else:
            key,value = line.split('=')
            layer[key] = value
    layers.append(layer)
    return layers

In [142]:
def create_model(layers_dict):
    model = Model()
    
    model_info = {}
    
    for layer_dict in layers_dict:
        if layer_dict['type'] == 'net':
            model_info = layer_dict
            height, width, channels = (int(model_info['height']), int(model_info['width']), int(model_info['channels']))
            input_layer = Input(( height, width, channels ))
            x = input_layer
        elif layer_dict['type'] == 'convolutional':
            filters = int(layer_dict['filters'])
            kernel_size = (int(layer_dict['size']), int(layer_dict['size']))
            padding = 'same' if layer_dict['pad'] == '1' else 'valid'
            stride = (int(layer_dict['stride']), int(layer_dict['stride']))
            
            activation = layer_dict['activation']
            
            x = Conv2D(filters = filters, kernel_size = kernel_size, padding = padding, strides = stride)(x)
            
            try:
                batch_norm = int(layer_dict['batch_normalize'])
                if batch_norm:
                    x = BatchNormalization()(x)
            except:
                pass
            
            if activation == 'leaky':
                x = LeakyReLU(0.1)(x)
            elif activation == 'relu':
                x = Activation(activation='linear')(x)
        elif layer_dict['type'] == 'maxpool':
            size = ( int(layer_dict['size']), int(layer_dict['size']) )
            stride = int(layer_dict['stride'])
            padding = 'same' if stride == 1 else 'valid'
    
            x = MaxPool2D(size,strides=stride,padding=padding)(x)
        elif layer_dict['type'] == 'yolo':
            break
            
    model = Model(inputs=input_layer,outputs = x)
    return model

In [143]:
model = create_model(parse_cfg(model_cfg))

In [144]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 416, 416, 3)       0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 416, 416, 16)      448       
_________________________________________________________________
batch_normalization_75 (Batc (None, 416, 416, 16)      64        
_________________________________________________________________
leaky_re_lu_75 (LeakyReLU)   (None, 416, 416, 16)      0         
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 208, 208, 16)      0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 208, 208, 32)      4640      
_________________________________________________________________
batch_normalization_76 (Batc (None, 208, 208, 32)      128       
__________

In [104]:
weight = np.fromfile('yolov3-tiny.weights',dtype=np.float32)

In [105]:
weight.shape

(8858739,)

In [109]:
weight[4]

0.0

In [126]:
len(model.layers[6].get_weights()[3])

32

3782.7999999999997